In [126]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata
from torchaudio import transforms

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

# for one-hot vectore
import torch.nn.functional as func

from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA

In [127]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data/data") #need to change this back before pushing ***
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*5) for data in data_array]
print(len(data_array[0]))

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_1337/3951241112.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


240000


In [128]:
#implementing the Mel-frequency cepstrum coefficients as feature vectors
mfcc = transforms.MFCC(sample_rate=48000, n_mfcc=40, melkwargs={"n_fft": 100, "hop_length": 140, "n_mels": 44, "center": False})
#print(train_ten.shape)
#mel_spectrogram = transforms.MelSpectrogram(sample_rate=48000, n_fft=400, n_mels=1)
#print("2", mel_spectrogram(train_ten[0].float()).shape)
#print("2.5", mfcc(mel_spectrogram(train_ten[0].float())).shape)
#print(mfcc(train_ten[0].float()).shape)
#mfcc_features = np.zeros((562, 48040))
#for row in range(562):
#    mel_coef = np.array(mfcc(train_ten[row].float()))
#    mel_coef = mel_coef.reshape((1, 48040))
#    mfcc_features[row] = mel_coef
mfcs = transforms.MelSpectrogram(sample_rate=48000, n_fft=100)

/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (44) may be set too high. Or, the value for `n_freqs` (51) may be set too low.
  warnings.warn(
/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (51) may be set too low.
  warnings.warn(


In [129]:
# convolution layer
# structure comes from IBM website
# will use mel_coef as feautre vectors

#mfcc_features = torch.tensor(mfcc_features)

CONV_KERNEL = 3
conv_layer1 = torch.nn.Conv2d(1, 1, (CONV_KERNEL, CONV_KERNEL))
#conv_layer2 = torch.nn.Conv2d(1, 1, (CONV_KERNEL, CONV_KERNEL))
#conv_layer3 = torch.nn.Conv1d(50, 50, CONV_KERNEL)
#conv_layer2 = torch.nn.
#conv_layer3 = 
#conv_out = conv_layer(mfcc_features)

#print(conv_out.shape)



In [154]:
# pooling layer
KERNEL_SIZE = 3
pooling_layer = torch.nn.MaxPool2d(KERNEL_SIZE)

#pooled_out = pooling_layer(conv_out)

#pooled_out = pooled_out.flatten()
#pooled_out.shape

In [158]:
# fully connected layer
#implementing kernel SVM
class MLP(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(in_features=67158, out_features=1000)
    self.fc2 = torch.nn.Linear(in_features=1000, out_features=100)
    self.fc3 = torch.nn.Linear(in_features=100, out_features=8)
    #self.fc4 = torch.nn.Linear(in_features=10, out_features=8)

  def forward(self, x):
    x1 = torch.relu(self.fc1(x))
    x2 = torch.relu(self.fc2(x1))
    x3 = self.fc3(x2)
    #x4 = self.fc4(x3) #usually left linear
    return torch.sigmoid(x3) #for classification

In [132]:
mfcs_ten = mfcs(torch.tensor(data_array[0]).float())
print(mfcs_ten.shape)
print(mfcs_ten)

torch.Size([128, 4801])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.8382,  4.5661, 22.2328,  ..., 18.6454, 23.0321,  8.8485],
        [ 0.1564,  4.2948, 11.4627,  ..., 27.4101, 16.7488, 12.5306],
        [ 1.2654,  2.5641,  6.4479,  ..., 21.8701, 18.7278, 79.4418]])


In [159]:
# split into training and testing
#transformer = KernelPCA(n_components=40, kernel='cosine')
mfcs_data = []
for data in data_array:
    mfcs_image = mfcs(torch.tensor(data).float())
    mfcs_data.append(np.array(mfcs_image))
#mfcc_data_array = [np.array(mfcs(torch.tensor(data).float())) for data in data_array] #transform into mfcc before model
train_data, test_data, train_labels, test_labels = train_test_split(mfcs_data, label_array, train_size=0.75, test_size=0.25)
#train_data = mfcc_data_array
#train_labels = label_array
#print(data.shape, train_data.shape, test_data.shape)
for i in range(len(train_data)):
    train_data[i] = train_data[i].reshape(1, 128, 4801)
for i in range(len(test_data)):
    test_data[i] = test_data[i].reshape(1, 128, 4801)
train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)
print(train_ten.shape)
train_dataset = tdata.TensorDataset(train_ten, train_y_ten)

train_loader = tdata.DataLoader(train_dataset, batch_size=50) #this is the data loader (just doing 1 point at a time)

torch.Size([843, 1, 128, 4801])


In [160]:
print(train_ten.shape)
print(test_ten.shape)
print(train_y_ten.shape)
print(test_y_ten.shape)

torch.Size([843, 1, 128, 4801])
torch.Size([282, 1, 128, 4801])
torch.Size([843])
torch.Size([282])


In [161]:
n_epochs = 10 #essentially number of iterations

#instantiating a model
model = MLP()

#loss function
criterion = torch.nn.CrossEntropyLoss() #binary cross-entropy loss, clamps log values to protect against extremes

#optimizer mechanism
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())

loss_array = np.zeros(n_epochs)
#something about test accuracy here - see final code
for epoch_idx in range(n_epochs):
  print(epoch_idx)
  for batch in train_loader:
    data_batch, labels_batch = batch
    # convolutional layer
    #data_batch = data_batch[None, ...]
    conv_out = conv_layer1(data_batch)
    
    #conv_out = conv_layer1(conv_out)
    #conv_out = conv_layer3(conv_out)
    relu_conv = torch.relu(conv_out)
    #print("1", conv_out.shape)
    # pooling layer
  
    pooled_out = pooling_layer(relu_conv)
    #print("2", pooled_out.shape)
    fc_layer_in = pooled_out.reshape((pooled_out.shape[0], -1))
    # move to GPU if available - data_batch = data_batch.cuda() etc.
    #predict
    #print("3",fc_layer_in.shape)
    labels_pred = model(fc_layer_in.to(torch.float32))
   
    #print(labels_pred.shape)
    #compute loss
    loss = criterion(labels_pred, labels_batch) #note: cross entropy is not symmetric

    loss_array[epoch_idx] += loss.item()/data_batch.shape[0] #mean of loss
    #plotting the precision
    #with torch.no_grad():
      #test_pred = model(test_ten).flatten()
      #test_loss =
 
    #backprop
    optimizer.zero_grad()
    loss.backward() #backprop from pytorch
    optimizer.step() #all params optimized
  #if epoch_idx%50 == 0:
  #  fig.data[0].y = loss_array[:epoch_idx]
  #  fig.update_yaxes(type='log')

0
1
2
3
4
5
6
7
8
9


In [81]:
# loading the raw data for feature learning
test_path = os.path.join(os.getcwd(), "..", "test_data")
test_files = os.listdir(test_path)
test_wav_files = []

for file in test_files:
    curr_path = os.path.join(test_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        test_wav_files.append(file)

test_data_array = []
for data in test_wav_files: #in the my_datasets code
    test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])

test_data_array = [np.resize(data, 48000*5) for data in test_data_array]

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_87777/2716096835.py:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])


In [84]:
# split into training and testing
mfcc_test_data_array = [np.array(mfcc(torch.tensor(data).float())) for data in test_data_array] #transform into mfcc before model
test_data = mfcc_test_data_array
#print(data.shape, train_data.shape, test_data.shape)

test_ten = torch.tensor(test_data)
test_dataset = tdata.TensorDataset(test_ten)

test_loader = tdata.DataLoader(test_dataset, batch_size=50)

In [163]:
test_pred = []
test_dataset = tdata.TensorDataset(test_ten, test_y_ten)
test_loader = tdata.DataLoader(test_dataset, batch_size=50) 
with torch.no_grad():
  for test in test_loader:
    # convolutional layer
    if test[0].shape[0] != 50:
      break
    conv_test = conv_layer1(test[0])
    #conv_test = conv_layer2(conv_test)
    #conv_test = conv_layer3(conv_test)
    relu_conv_test = torch.relu(conv_test)

    # pooling layer
    pooled_test = pooling_layer(relu_conv_test)
    fc_layer_test = pooled_test.reshape((pooled_test.shape[0], -1))
    
    label_histogram = model(fc_layer_test.to(torch.float32))
    for histogram in label_histogram:
      test_pred.append(np.argmax(histogram).item())

test_pred.extend([0]*32)
test_pred = torch.tensor(test_pred)
print(classification_report(test_y_ten, test_pred))

'''
LABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])

#counting the mislabeled points
with open("cnn_results.csv", "w+") as result_file:
    writer = csv.writer(result_file)
    writer.writerow(["filename", "label"])

    for idx in range(len(test_pred)):
        writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[test_pred[idx]]])

#print(classification_report(test_y_ten, test_pred))
#print("Number of mislabeled points out of a total %d points : %d", (test_ten.shape[0], (test_y_ten != y_pred).sum()))
'''

              precision    recall  f1-score   support

           0       0.08      0.23      0.12        22
           1       0.13      0.58      0.22        38
           2       0.14      0.06      0.08        36
           3       0.13      0.13      0.13        30
           4       0.00      0.00      0.00        40
           5       0.67      0.05      0.09        40
           6       0.33      0.05      0.09        38
           7       0.00      0.00      0.00        38

    accuracy                           0.13       282
   macro avg       0.19      0.14      0.09       282
weighted avg       0.20      0.13      0.09       282



'\nLABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])\n\n#counting the mislabeled points\nwith open("cnn_results.csv", "w+") as result_file:\n    writer = csv.writer(result_file)\n    writer.writerow(["filename", "label"])\n\n    for idx in range(len(test_pred)):\n        writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[test_pred[idx]]])\n\n#print(classification_report(test_y_ten, test_pred))\n#print("Number of mislabeled points out of a total %d points : %d", (test_ten.shape[0], (test_y_ten != y_pred).sum()))\n'